In [40]:
import datetime
import pandas as pd

import pickle

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing

In [41]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [42]:
MODEL_FOLDER = "model"

PROCESSED_DATA_FOLDER = "data/4_all_data_preprocessed"
ISW_DATA_FILE = "all_isw.csv"
WEATHER_EVENTS_DATA_FILE = "features_generated_v1.csv"

OUTPUT_DATA_FILE = "all_features"

tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"

tfidf_transformer_version = "v4"
count_vectorizer_version = "v4"

In [43]:
def isNaN(num):
    return num != num

### Load preprocessed data

In [44]:
# Load CSVs
df_isw = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{ISW_DATA_FILE}", sep=";")
df_weather_events = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{WEATHER_EVENTS_DATA_FILE}", sep=";")

df_weather_events = df_weather_events.loc[:, ~df_weather_events.columns.str.contains('^Unnamed')]

print(df_isw.shape)

# Load TF-IDF
tfidf = pickle.load(open(f"{MODEL_FOLDER}/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", "rb"))
cv = pickle.load(open(f"{MODEL_FOLDER}/{count_vectorizer_model}_{count_vectorizer_version}.pkl", "rb"))

(402, 7)


### Prepare weather & events

In [45]:
# Technicaly it's feature engineering

# Replace timestamps with numerical values
df_weather_events.day_sunset = df_weather_events.day_sunset.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.day_sunrise = df_weather_events.day_sunrise.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.hour_datetime = df_weather_events.hour_datetime.apply(lambda x:
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()//3600
)

# Encode categories
le = preprocessing.LabelEncoder()
df_weather_events.hour_conditions = le.fit_transform(df_weather_events.hour_conditions)
df_weather_events.hour_preciptype = le.fit_transform(df_weather_events.hour_preciptype)

# Create new column with boolean event indicator
df_weather_events["event_indicator"] = df_weather_events.event_hour_level_event_datetimeEpoch.apply(lambda x: not isNaN(x))

# Drop now irrelevant features
features_to_drop = [
    "day_datetimeEpoch",
    "hour_datetimeEpoch",
    "event_all_region",
    "event_clean_end",
    "event_intersection_alarm_id",
    "event_start_time",
    "event_end_time",
    "event_start_hour",
    "event_end_hour",
    "event_day_date",
    "event_start_hour_datetimeEpoch",
    "event_end_hour_datetimeEpoch",	
    "event_hour_level_event_time",
    "event_hour_level_event_datetimeEpoch",
    "event_region_title",
    "event_region_city",
    "event_start",
    "event_end",
    "city",
    "region",
    "center_city_ua",
    "center_city_en",
    "region_alt"
]
df_weather_events = df_weather_events.drop(features_to_drop, axis=1)

# Convert everything to float
df_weather_events.hour_preciptype = df_weather_events.hour_preciptype.astype(float)
df_weather_events.hour_conditions = df_weather_events.hour_conditions.astype(float)
df_weather_events.region_id = df_weather_events.region_id.astype(float)
df_weather_events.event_alarms_past_24 = df_weather_events.event_alarms_past_24.astype(float)
df_weather_events.event_simultaneous_alarms = df_weather_events.event_simultaneous_alarms.astype(float)

df_weather_events.event_holiday_is_near = df_weather_events.event_holiday_is_near.apply(lambda x: 1.0 if x and not isNaN(x) else 0.0)

# Fill NAN
df_weather_events.fillna(0, inplace=True)

print(df_weather_events.dtypes)
df_weather_events.sample(10)

city_resolvedAddress            object
day_datetime                    object
day_tempmax                    float64
day_tempmin                    float64
day_temp                       float64
day_dew                        float64
day_humidity                   float64
day_precip                     float64
day_precipcover                float64
day_solarradiation             float64
day_solarenergy                float64
day_uvindex                    float64
day_sunrise                    float64
day_sunset                     float64
day_moonphase                  float64
hour_datetime                  float64
hour_temp                      float64
hour_humidity                  float64
hour_dew                       float64
hour_precip                    float64
hour_precipprob                float64
hour_snow                      float64
hour_snowdepth                 float64
hour_preciptype                float64
hour_windgust                  float64
hour_windspeed           

,city_resolvedAddress,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_sunrise,day_sunset,day_moonphase,hour_datetime,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,region_id,event_alarms_past_24,event_simultaneous_alarms,event_hours_from_last_alarm,event_holiday_is_near,event_indicator
46452,"Чернівці, Україна",2022-04-30,16.9,6.8,11.8,4.6,64.5,0.000,0.00,243.0,20.9,8.0,21564.0,73715.0,0.97,9.0,8.7,79.18,5.3,0.000,0.0,0.0,0.0,5.0,9.2,5.1,252.0,1025.6,15.8,70.5,357.0,1.3,4.0,10.0,4.0,24.0,1.0,0.0,9.340833,0.0,False
49852,"Чернівці, Україна",2022-09-18,13.8,8.3,11.4,8.6,83.7,20.534,8.33,30.9,2.7,1.0,25018.0,69787.0,0.73,17.0,13.8,61.92,6.6,0.000,0.0,0.0,0.0,5.0,61.6,21.2,296.0,1003.7,13.0,83.0,51.0,0.2,1.0,10.0,4.0,24.0,0.0,0.0,23.397222,0.0,False
165530,"Івано-Франківськ, Україна",2022-07-01,33.0,16.6,25.7,17.0,62.6,0.000,0.00,306.3,26.4,9.0,19284.0,77305.0,0.06,3.0,18.6,89.84,16.9,0.000,0.0,0.0,0.0,5.0,7.9,3.6,220.0,1015.1,10.0,0.0,0.0,0.0,0.0,10.0,0.0,9.0,0.0,0.0,14.958333,1.0,False
106816,"Ужгород, Ужгородський район, Україна",2022-05-24,21.3,6.7,14.6,7.2,63.4,0.000,0.00,244.2,21.2,8.0,20391.0,76579.0,0.78,18.0,20.6,45.62,8.5,0.000,0.0,0.0,0.0,5.0,19.8,7.8,224.0,1010.2,42.3,57.7,164.0,0.6,2.0,10.0,4.0,7.0,1.0,0.0,3.964444,0.0,True
21306,"Дніпро, Україна",2022-07-28,29.7,18.5,24.8,16.9,63.4,1.168,4.17,261.1,22.6,9.0,18546.0,73385.0,0.97,20.0,26.8,58.18,17.9,0.000,0.0,0.0,0.0,5.0,25.9,8.6,349.4,1016.0,24.1,0.5,48.0,0.2,0.0,10.0,0.0,4.0,4.0,2.0,4.537500,0.0,False
185883,"Суми, Україна",2022-12-06,-3.1,-10.8,-7.4,-14.5,58.0,0.000,0.00,37.5,3.3,2.0,26866.0,56138.0,0.41,0.0,-8.8,66.01,-14.0,0.000,0.0,0.0,0.0,5.0,19.1,9.4,136.5,1038.0,24.1,35.0,0.0,0.0,0.0,10.0,4.0,18.0,1.0,0.0,7.785556,1.0,False
128214,"Рівне, Україна",2022-12-11,0.6,-2.2,-0.3,-1.0,95.2,10.300,54.17,2.4,0.0,0.0,29170.0,58208.0,0.58,4.0,-0.2,95.03,-0.9,1.000,100.0,0.4,1.8,3.0,24.8,12.6,41.5,1003.0,1.6,100.0,0.0,0.0,0.0,10.0,6.0,17.0,0.0,0.0,23.982778,0.0,False
47475,"Чернівці, Україна",2022-06-11,23.3,15.0,18.3,15.8,85.7,24.508,41.67,212.6,18.3,8.0,18929.0,76601.0,0.38,21.0,16.0,96.67,15.5,18.111,100.0,0.0,0.0,3.0,30.7,7.7,310.0,1015.5,47.2,99.9,13.0,0.0,0.0,10.0,6.0,24.0,0.0,0.0,19.257500,0.0,False
95047,"Хмельницька область, Україна",2022-11-13,8.8,6.6,7.5,6.5,93.9,0.000,0.00,35.2,3.1,3.0,26369.0,59561.0,0.63,1.0,7.3,94.01,6.4,0.000,0.0,0.0,0.0,5.0,14.4,7.6,287.7,1033.0,24.1,84.5,0.0,0.0,0.0,10.0,4.0,22.0,1.0,0.0,9.068056,0.0,False
73693,"Миколаїв, Україна",2022-07-03,29.5,18.6,24.3,10.3,43.4,0.000,0.00,344.2,29.8,9.0,18139.0,75003.0,0.13,20.0,28.4,28.49,8.4,0.000,0.0,0.0,0.0,5.0,18.7,4.3,324.8,1017.0,24.1,93.2,145.0,0.5,1.0,10.0,3.0,14.0,10.0,0.0,2.560278,1.0,True


### Calculate TF-IDF matrix

In [46]:
# Calculate word count vector for each report (save as matrix)
word_count_vector = cv.transform(df_isw["lemm"].values.astype("U"))
# print("Word count vector:\n", word_count_vector)

# Calculate global tfidf vectors matrix
tfidf_vector = tfidf.transform(word_count_vector)

# transform tfidf matrix to sparse dataframe
df_tfidf_vector = pd.DataFrame.sparse.from_spmatrix(tfidf_vector, columns=cv.get_feature_names_out())
df_tfidf_vector = pd.concat([df_isw.date_tomorrow_datetime, df_tfidf_vector], axis=1)

print(df_tfidf_vector.shape)

(402, 7282)


### Merge final dataset

In [48]:
# Merge weather events dataset with yesterday report tfidf matrix (takes 2m to execute)
df_final = df_weather_events.merge(df_tfidf_vector.add_prefix("isw_"),
                                    how="inner",
                                    left_on="day_datetime",
                                    right_on="isw_date_tomorrow_datetime")

# Reorder columns
df_final.insert(2, 'isw_date_tomorrow_datetime', df_final.pop('isw_date_tomorrow_datetime'))
df_final.insert(3, 'event_indicator', df_final.pop('event_indicator'))

print(df_final.shape)
df_final.sample(5)

(192867, 7323)


city_resolvedAddress day_datetime isw_date_tomorrow_datetime  \
148027              Донецьк, Україна   2022-10-30                 2022-10-30   
171092  Хмельницька область, Україна   2022-12-11                 2022-12-11   
121386  Хмельницька область, Україна   2022-09-14                 2022-09-14   
53196   Хмельницька область, Україна   2022-05-19                 2022-05-19   
17046                  Київ, Україна   2022-03-21                 2022-03-21   

        event_indicator  day_tempmax  day_tempmin  day_temp  day_dew  \
148027            False         14.4          9.3      11.3      8.5   
171092            False          6.1         -1.3       2.5      1.8   
121386            False         20.4          8.2      14.1     10.1   
53196             False         19.7          5.2      12.4      2.3   
17046              True         12.9         -0.8       5.9     -3.7   

        day_humidity  day_precip  day_precipcover  day_solarradiation  \
148027          83.0         2.4            37.50                29.0   
171092          95.1         4.9            25.00                 2.8   
121386          78.4         0.8             8.33               134.0   
53196           53.8         0.0             0.00               319.0   
17046           51.8         0.0             0.00               177.9   

        day_solarenergy  day_uvindex  day_sunrise  day_sunset  day_moonphase  \
148027              2.6          2.0      22212.0     58436.0           0.15   
171092              0.1          0.0      28667.0     58364.0           0.58   
121386             11.7          6.0      24391.0     70083.0           0.59   
53196              27.7          9.0      19412.0     75272.0           0.61   
17046              15.4          5.0      21532.0     65539.0           0.62   

        hour_datetime  hour_temp  hour_humidity  hour_dew  hour_precip  \
148027           12.0       13.1          83.12      10.3          0.0   
171092           15.0        5.4          98.62       5.2          0.0   
121386            9.0       11.1          88.07       9.2          0.1   
53196            15.0       18.1          37.05       3.2          0.0   
17046             4.0       -0.4          72.17      -4.8          0.0   

        hour_precipprob  hour_snow  hour_snowdepth  hour_preciptype  \
148027              0.0        0.0             0.0              5.0   
171092              0.0        0.0             1.8              5.0   
121386            100.0        0.0             0.0              3.0   
53196               0.0        0.0             0.0              5.0   
17046               0.0        0.0             0.0              5.0   

        hour_windgust  hour_windspeed  hour_winddir  hour_pressure  \
148027           52.2            21.2         271.0         1012.0   
171092           16.9             8.3         249.4          992.0   
121386           17.3            10.8         230.0         1008.7   
53196            32.0            14.0         345.1         1025.0   
17046            16.6             6.5         325.9         1039.0   

        hour_visibility  hour_cloudcover  hour_solarradiation  \
148027             24.1            100.0                 75.0   
171092              0.2            100.0                  4.0   
121386             10.0            100.0                135.0   
53196              24.1             26.9                859.0   
17046              24.1              0.0                  0.0   

        hour_solarenergy  hour_uvindex  hour_severerisk  hour_conditions  \
148027               0.3           1.0             10.0              3.0   
171092               0.0           0.0             10.0              3.0   
121386               0.5           1.0             10.0              6.0   
53196                3.1           9.0             10.0              4.0   
17046                0.0           0.0             10.0              0.0   

        region_id  event_alarms_past_24  

In [51]:
# Save final dataset
with open(f"{PROCESSED_DATA_FOLDER}/{OUTPUT_DATA_FILE}.pkl", 'wb') as handle:
    pickle.dump(df_final, handle)

In [10]:
"""y = df_final.event_indicator.astype(float)
df_final = df_final.drop(["isw_date_tomorrow_datetime", "day_datetime", "city_resolvedAddress", "event_indicator"], axis=1)

y.head(5)"""

'y = df_final.event_indicator.astype(float)\ndf_final = df_final.drop(["isw_date_tomorrow_datetime", "day_datetime", "city_resolvedAddress", "event_indicator"], axis=1)\n\ny.head(5)'